In [1]:
import pandas as pd
import data_util
from tqdm.notebook import tqdm
#from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from tec_an import *
import numpy as np
from data_util import *
from sklearn_model_hyper import *
from joblib import dump, load
from data_generator import *
from data_agent import *
from stock_agent import *
from backtest import *
from bitstamp import *

from tqdm.notebook import tqdm

import math
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as scs
import scikitplot as skplt

from keras.wrappers.scikit_learn import KerasClassifier
from imblearn.over_sampling import RandomOverSampler
import random
from catboost import CatBoost

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.utils.validation import check_is_fitted

from joblib import dump, load
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import *
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from pytorch_tabnet.tab_model import TabNetClassifier

from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE, RFECV

path = "./data/"


In [2]:
#train_start_list = [1622502000, 1590966000, 1559343600, 1580515200, 1612137600]
train_start_list = [1622502000, 1590966000, 1559343600, 1527807600, 1517443200, 1548979200, 1580515200, 1612137600]
#train_start_list = [1612137600]
train_keys = ["btcusd", "ethusd", "bchbtc"]
#train_keys = ["bchbtc"]
val_start = 1626340500
val_end = 1626369600

In [3]:
load_from_disk = True

# '2Min'

In [4]:
def get_online_data(minutes, source_data_generator, load_from_disk, file_prefix = ""):
    
    online = OnLineDataProvider(
                 source_data_generator = source_data_generator,
                 minutes = minutes,
                 train_keys = train_keys,
                 train_limit = 1000,
                 val_limit = 1000,
                 val_keys = ["btcusd"],
                 val_start = val_start,
                 val_end = val_end,
                 train_start_list = train_start_list
    )

    online_path = f'data/online{file_prefix}_{minutes}'
    
    if (load_from_disk):
        online = load(online_path)    
    else:
        #online.load_train_cache()
        online.load_cache()
        online.sourceDataGenerator = None
        dump(online, online_path)
        
    
    return online

In [5]:
class TabNetClassifierEarly(TabNetClassifier):
    def fit(self, X_train, Y_train):
        super().fit(X_train, 
                    Y_train, 
                    eval_set=[(X_train, Y_train)],
                    eval_metric=['balanced_accuracy'])

In [6]:
%%time

def get_classifiers():
    clss = classifiers = [
        lambda : CatBoostClassifier(logging_level = 'Silent'),
        
        #lambda : make_pipeline(StandardScaler(),
        #                        MLPClassifier(
        #                            solver='lbfgs', alpha=1, random_state=1, max_iter=20000,
        #                            early_stopping=True)
        #                        ),
        
        #lambda : MLPClassifier(alpha=1, random_state=1, max_iter=20000, early_stopping=True),
        
        lambda : xgb.XGBClassifier(random_state=1,
                                   objective = "binary:logistic",
                                   eval_metric='mlogloss',
                                   learning_rate=0.01),
        
        #lambda : GaussianNB(),
        #lambda : QuadraticDiscriminantAnalysis(),
        #lambda : AdaBoostClassifier(random_state = 42),
        
        lambda : TabNetClassifierEarly(verbose=0),
        
        #lambda : KNeighborsClassifier(3),
        #lambda : GaussianProcessClassifier(1.0 * RBF(1.0)),
        
        
        lambda : DecisionTreeClassifier(random_state = 42),
        #lambda : RandomForestClassifier(max_depth=50, n_estimators=100, max_features=1),
        lambda : rand(),
        
        #lambda : SVC(gamma=2, C=1, random_state = 42),
    ]
    
    def rand():
        ##print("rand created")
        param = {'criterion': 'entropy', 'max_depth': 100, 'n_estimators': 30, 'random_state': 42}
        return RandomForestClassifier(**param)
    

    clss = []
    clss.append(lambda : rand())
    #clss.append(lambda : make_pipeline(StandardScaler(),TabNetClassifierEarly(verbose=0)))
    return clss

estimators = [
                lambda : RFE(estimator=RandomForestClassifier(random_state=10), n_features_to_select=10),
                lambda : RFE(estimator=DecisionTreeClassifier(random_state=10), n_features_to_select=8),
                lambda : RFECV(estimator=DecisionTreeClassifier(random_state=10)),
                lambda : RFECV(estimator=RandomForestClassifier(random_state=10))
                ]


scoring = [
    'accuracy', 
    'f1', 
    'precision', 
    'recall'
]

estimators = []

#for score in scoring:
    
estimators.append(lambda : RFECV(estimator=DecisionTreeClassifier(random_state=10),
                    scoring='accuracy'))    
estimators.append(lambda : RFECV(estimator=DecisionTreeClassifier(random_state=10),
                    scoring='f1'))    
estimators.append(lambda : RFECV(estimator=DecisionTreeClassifier(random_state=10),
                    scoring='precision'))    
estimators = []
estimators.append(lambda : RFECV(estimator=DecisionTreeClassifier(random_state=10),
                    scoring='recall'))

def add_normalizers(models, cls):
    models.append(make_pipeline(StandardScaler(),cls()))
    models.append(make_pipeline(MinMaxScaler(),cls()))
    models.append(make_pipeline(Normalizer(),cls()))


def get_all_models():
    models = []
    classifiers = get_classifiers()
    for cls in classifiers:
        add_normalizers(models, cls)
        #models.append(MockModel(cls()))
        #models.append(MockCoPilotModel(cls(), getModel()))
        
        for est in estimators:
            tmp = lambda : Pipeline(
                steps=[
                ('s',est()),
                ('m',cls())
            ])
            add_normalizers(models, tmp)
            #models.append(tmp())
            #models.append(MockModel(tmp()))
            #models.append(MockCoPilotModel(tmp(), getModel()))
    return models


CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 7.87 µs


In [7]:
get_all_models_factory = lambda : get_all_models()

In [8]:
%%time

def test_models(provider, get_all_models_factory, steps = [1]):
    
    score_board_by_step = {}

    for step in steps:
        print(f"Step {step}")
        models_list = get_all_models_factory()
        model_result = []
        for idx in tqdm(range(len(models_list))):
            model_creator = models_list[idx]
            model = model_creator
            #print(f"Trainning {model}")
            train_by_step(model, step, provider)
            
            result = run_trial(model, provider, step)

            result['step'] = step

            model_result.append(result)
        
        score_board_by_step[step] = model_result
        
    best_models = []

    best_model = []

    best_score = -1
    
    best_step = -1
    
    model_rank = []
    
    def myFunc(e):
        return e['profit']

    for step_idx in score_board_by_step:
        step_board = score_board_by_step[step_idx]
        for result in step_board:
            if (result['profit'] > 0):
                model_rank.append(result)
            
    model_rank.sort(key=myFunc, reverse = True)
    
    return model_rank

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.96 µs


In [9]:

steps = [1, 5]

windows = [5, 10, 20, 30, 40]
#windows = [40]

minutes = [15, 5, 3, 1]
#minutes = [3]


In [10]:
def process_model(minu, win, step, model, provider):
        #print(f"Training {model} {provider} ")
        train_by_step(model, step, provider)
        #print(f"Checking {minu} {win} {step} ")
        result = run_trial(model, provider, step)
        result['window'] = win
        result['minutes'] = minu
        result['step'] = step
        #print(result)
        return result

In [11]:
%%time

min_results = []

process_list = []

for minu in minutes:
    results = []
    for win in windows:
        tec = TecAn(windows = win, windows_limit = 100)
        sourceDataGenerator = SourceDataGenerator(tec = tec)
        provider = get_online_data(minu, sourceDataGenerator, load_from_disk, win)
        for step in steps:
            models_list = get_all_models_factory()
            for idx in range(len(models_list)):
                model_creator = models_list[idx]
                model = model_creator
                process = {}
                process['window'] = win
                process['minutes'] = minu
                process['step'] = step
                process['model'] = model
                process['provider'] = provider
                process_list.append(process)

for process in tqdm(process_list):
    win = process['window'] 
    minu = process['minutes']
    step = process['step'] 
    model = process['model'] 
    
    tec = TecAn(windows = win, windows_limit = 100)
    sourceDataGenerator = SourceDataGenerator(tec = tec)
    provider = get_online_data(minu, sourceDataGenerator, load_from_disk, win)
    
    result = process_model(minu, win, step, model, provider)
    min_results.append(result)

  0%|          | 0/240 [00:00<?, ?it/s]

CPU times: user 9h 49min 8s, sys: 3min 15s, total: 9h 52min 23s
Wall time: 9h 53min 13s


In [12]:
model_path = f'model/all_results'
dump(min_results, model_path)

['model/all_results']

In [13]:
best_results = []
for win_result in min_results:
    profit = win_result['profit']
    model = win_result['model']
    window = win_result['window']
    minutes = win_result['minutes']
    step = win_result['step']
    profit = win_result['profit']
    models_score = win_result['models_score']
    #print(models_score)
    if (profit > 100):
        print(f"Minutes={minutes} Window={window} Step={step} | {profit}")
        print(f"{win_result['model']}")
        print(f"")
        best_results.append(win_result)

print(f"{len(best_results)} selected")

Minutes=15 Window=5 Step=1 | 110.6958224494712
Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(criterion='entropy', max_depth=100,
                                        n_estimators=30, random_state=42))])

Minutes=15 Window=5 Step=1 | 100.97476421422066
Pipeline(steps=[('normalizer', Normalizer()),
                ('randomforestclassifier',
                 RandomForestClassifier(criterion='entropy', max_depth=100,
                                        n_estimators=30, random_state=42))])

Minutes=15 Window=5 Step=1 | 100.22608456616095
Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('pipeline',
                 Pipeline(steps=[('s',
                                  RFECV(estimator=DecisionTreeClassifier(random_state=10),
                                        scoring='recall')),
                                 ('m',
                                  RandomForestClassifier(cr

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pipeline',
                 Pipeline(steps=[('s',
                                  RFECV(estimator=DecisionTreeClassifier(random_state=10),
                                        scoring='recall')),
                                 ('m',
                                  RandomForestClassifier(criterion='entropy',
                                                         max_depth=100,
                                                         n_estimators=30,
                                                         random_state=42))]))])

Minutes=5 Window=30 Step=1 | 100.87610794254047
Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('pipeline',
                 Pipeline(steps=[('s',
                                  RFECV(estimator=DecisionTreeClassifier(random_state=10),
                                        scoring='recall')),
                                 ('m',
                           

                                                         random_state=42))]))])

Minutes=1 Window=30 Step=1 | 100.35436705185553
Pipeline(steps=[('normalizer', Normalizer()),
                ('pipeline',
                 Pipeline(steps=[('s',
                                  RFECV(estimator=DecisionTreeClassifier(random_state=10),
                                        scoring='recall')),
                                 ('m',
                                  RandomForestClassifier(criterion='entropy',
                                                         max_depth=100,
                                                         n_estimators=30,
                                                         random_state=42))]))])

Minutes=1 Window=40 Step=1 | 100.10774417597531
Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(criterion='entropy', max_depth=100,
                                        n_est

In [62]:
model_path = f'model/RandomForestClassifier'
dump(best_results, model_path)

['model/RandomForestClassifier']

In [32]:
def print_result(result):
    model = result['model']
    window = result['window']
    minutes = result['minutes']
    step = result['step']
    profit = result['profit']
    print(f"Minutes={minutes} Window={window} Step={step} | {profit} {result['models_profit_metric']}")
    print(f"{model}")

In [33]:
def myFunc(e):
    return e['profit']
    #return e['models_profit_metric']['btcusd']

best_results.sort(key=myFunc, reverse = False)

for best in best_results:
    print_result(best)
    model = best['model']
    window = best['window']
    minutes = best['minutes']
    step = best['step']
    model_path = f'model/result_{window}_{minutes}_{step}'
    dump(best, model_path)
    print(f"")

Minutes=1 Window=20 Step=1 | 100.00178533789413 {'ltceur': 0.004443746212716296, 'ltcusd': 0.014467608458106991, 'ethusd': 0.001578433722336029, 'btcusd': -0.001798166415155455}
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pipeline',
                 Pipeline(steps=[('s',
                                  RFECV(estimator=DecisionTreeClassifier(random_state=10),
                                        scoring='recall')),
                                 ('m',
                                  RandomForestClassifier(criterion='entropy',
                                                         max_depth=100,
                                                         n_estimators=30,
                                                         random_state=42))]))])

Minutes=1 Window=30 Step=1 | 100.05813131508344 {'ltceur': -0.000307116963132971, 'ltcusd': -0.00475464988309201, 'ethusd': 0.003928869844103186, 'btcusd': -0.0024142049092364904}
Pipeline(steps=[('normali


Minutes=1 Window=30 Step=1 | 100.35436705185553 {'ltceur': 0.005404077332051317, 'ltcusd': -0.02450955433969506, 'ethusd': 0.003281341244923389, 'btcusd': 0.0013062437847467657}
Pipeline(steps=[('normalizer', Normalizer()),
                ('pipeline',
                 Pipeline(steps=[('s',
                                  RFECV(estimator=DecisionTreeClassifier(random_state=10),
                                        scoring='recall')),
                                 ('m',
                                  RandomForestClassifier(criterion='entropy',
                                                         max_depth=100,
                                                         n_estimators=30,
                                                         random_state=42))]))])

Minutes=3 Window=10 Step=1 | 100.35688225883602 {'ltceur': -0.012178491216615331, 'ltcusd': 0.0007465719364258291, 'ethusd': -0.010966986041477094, 'btcusd': 0.027316552880410887}
Pipeline(steps=[('normalizer', N

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pipeline',
                 Pipeline(steps=[('s',
                                  RFECV(estimator=DecisionTreeClassifier(random_state=10),
                                        scoring='recall')),
                                 ('m',
                                  RandomForestClassifier(criterion='entropy',
                                                         max_depth=100,
                                                         n_estimators=30,
                                                         random_state=42))]))])

Minutes=3 Window=40 Step=1 | 101.33687848896164 {'ltceur': 0.004246749290570677, 'ltcusd': 0.035508259682388826, 'ethusd': 0.029098197346274852, 'btcusd': -0.0036932216247034556}
Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(criterion='entropy', max_depth=100,
                                   


Minutes=15 Window=10 Step=1 | 104.79654865922687 {'ltceur': 0.018742083870775306, 'ltcusd': -0.01555970648437968, 'ethusd': 0.012681106757985406, 'btcusd': 0.02509330434153996}
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(criterion='entropy', max_depth=100,
                                        n_estimators=30, random_state=42))])

Minutes=15 Window=20 Step=1 | 105.18823929991636 {'ltceur': -0.004681837989708759, 'ltcusd': 0.027842665929255116, 'ethusd': 0.017349792938575585, 'btcusd': -0.003988497780020516}
Pipeline(steps=[('normalizer', Normalizer()),
                ('randomforestclassifier',
                 RandomForestClassifier(criterion='entropy', max_depth=100,
                                        n_estimators=30, random_state=42))])

Minutes=5 Window=10 Step=1 | 105.3214336664476 {'ltceur': 0.0055075364069101206, 'ltcusd': 0.0030191878334797947, 'ethusd': 0.028994228824053503, 'b

In [34]:
def myFunc(e):
    return e['profit']
    #return e['models_profit_metric']['btcusd']

best_results.sort(key=myFunc, reverse = True)

In [35]:
best_results[0]

{'profit': 110.6958224494712,
 'model': Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                 ('randomforestclassifier',
                  RandomForestClassifier(criterion='entropy', max_depth=100,
                                         n_estimators=30, random_state=42))]),
 'models_profit': {'ltceur': 21.2252,
  'ltcusd': 8.8684,
  'ethusd': 6.2654,
  'btcusd': 6.4243},
 'models_score': {'ltceur': {'recall': 0.15982142857142856,
   'precision': 0.4398034398034398,
   'f1': 0.2344466273739358,
   'accuracy': 0.5945195976413458,
   'roc_auc': 0.5152482071955271},
  'ltcusd': {'recall': 0.16171328671328672,
   'precision': 0.42045454545454547,
   'f1': 0.23358585858585862,
   'accuracy': 0.5789108567464447,
   'roc_auc': 0.5075386445067296},
  'ethusd': {'recall': 0.18524871355060035,
   'precision': 0.4227005870841487,
   'f1': 0.25760286225402507,
   'accuracy': 0.5681581685744017,
   'roc_auc': 0.506718707386832},
  'btcusd': {'recall': 0.1750936329588015,
   'precision

In [19]:
def load_online(minutes, window, val_end, currency = []):
    tec = TecAn(windows = window, windows_limit = 100)
    source_data_generator = SourceDataGenerator(tec = tec)

    start = val_end - (60 * 100 * minutes)
    end = val_end - (160 * minutes)

    online = OnLineDataProvider(
                 source_data_generator = source_data_generator,
                 minutes = minutes,
                 train_keys = [],
                 train_limit = 40,
                 val_limit = 999,
                 val_keys = currency,
                 val_start = start,
                 val_end = end,
                 train_start_list = []
    )

    online.load_cache()
    return online

In [20]:
def backtest_model(model, x, closed_prices, back):
    
    for idx in range(len(x)):
        xx = np.array([x[idx]])
        #print(xx)
        yy = model.predict(xx)
        yy = yy[0]
        price = closed_prices[idx]
        #print(f'{idx} {yy} {price}')
        back.on_state(0, price)
        if(yy == 1):
            back.request_buy(price)
        else:
            back.request_sell(price)
    
    #print(f"Closing backtest_model {price} {back.current}")
    back.sell(price)
    #print(f"Closed backtest_model {back.current}")
    
    return back

In [21]:
def test_model(model, set_key, provider, step, verbose = True):
    valX, valY = provider.load_val_data(set_key)

    x, y, closed_prices = get_sequencial_data(valX, valY, step)
    
    #print(len(x))
    preds = model.predict(x)

    recall = recall_score(y, preds)
    precision = precision_score(y, preds)
    f1 = f1_score(y, preds)
    accuracy = accuracy_score(y, preds)

    back = BackTest(value = 100, 
                    verbose = verbose, 
                    sell_on_profit = True,
                    pending_sell_steps = step)
    back = backtest_model(model, x, closed_prices, back)
    return back

In [22]:
from datetime import datetime


timestamp = int(datetime.timestamp((datetime.now())))



In [70]:
def load_results(results_path):
    try:
        saved_models = load(results_path)
        print(f"{results_path} - {len(saved_models)}")
    except:
        saved_models = []
    return saved_models

In [73]:
decision = load_results("model/decision")
catboost = load_results("model/catboost")
randomForestClassifier = load_results("model/RandomForestClassifier")
models_results = [decision, catboost, randomForestClassifier]

model/decision - 280
model/catboost - 280
model/RandomForestClassifier - 58


In [92]:
all_models = []
for results in models_results:
    for result in results:
        if(result['profit'] > 100):
            all_models.append(result)
    
print(len(all_models))


165


In [99]:
online_cache = {}

In [100]:
scoreboard = []

fails = []

currency = "ltcusd"
currency = "ethusd"
currency = "batusd"

#currs = ["batusd", "ltcusd", "ethusd", "btcusd"]
currs = ["ethusd"]

#currs = online.val_keys

for best in tqdm(all_models):
    model = best['model']
    window = best['window']
    minutes = best['minutes']
    step = best['step']
    profit = best['profit']
    print(f"Minutes={minutes} Window={window} Step={step} | {profit}")
    cache_key = f"{minutes}-{window}"
    try:
        online = online_cache[cache_key]
    except :
        print(f"Not found {cache_key}")
        online = load_online(minutes = minutes, window = window, val_end = timestamp, currency = currs)
        online_cache[cache_key] = online
        
    profits = []
    backs = {}
    score = {}
    
    has_negative = False
    
    for train_set in currs:
        back = test_model(model, train_set, online, step, False)
        back_profit = back.get_profit()
        profits.append(back_profit)
        backs[train_set] = back
        print(f"{train_set} - {back_profit}")
        if (back_profit <= 0 and not has_negative):
            has_negative = True
        
    score['profit'] = np.average(profits)
    score['backs'] = backs
    score['result'] = best
    if (not has_negative):
        scoreboard.append(score)



  0%|          | 0/165 [00:00<?, ?it/s]

Minutes=15 Window=5 Step=1 | 103.42841714934886
Not found 15-5
https://www.bitstamp.net/api/v2/ohlc/ethusd/?&currency_pair=ethusd&step=900&limit=1000&start=1627334016
Downloaded from 2021-07-26 22:00:00 to 2021-07-27 22:45:00 2264.85
Resample 15Min - TecAn ( windows 5, windows_limit 100 )
Processing 99 of ethusd


  0%|          | 0/99 [00:00<?, ?it/s]

ethusd - 0.8235
Minutes=15 Window=10 Step=1 | 101.70597967714077
Not found 15-10
https://www.bitstamp.net/api/v2/ohlc/ethusd/?&currency_pair=ethusd&step=900&limit=1000&start=1627334016
Downloaded from 2021-07-26 22:00:00 to 2021-07-27 22:45:00 2264.85
Resample 15Min - TecAn ( windows 10, windows_limit 100 )
Processing 99 of ethusd


  0%|          | 0/99 [00:00<?, ?it/s]

...

ethusd - 0.0
Minutes=1 Window=30 Step=1 | 100.05813131508344


/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ethusd - 0.0
Minutes=1 Window=20 Step=1 | 100.00178533789413


/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ethusd - 0.0


In [101]:
profs = []

print(f"Selected: {len(scoreboard)}")

scoreboard.sort(key=myFunc, reverse = True)

for score in scoreboard:
    profit = score['profit']
    result = score['result']
    backs = score['backs']
    print(f"Current profit: {profit}")
    print_result(result)
    for back in backs:
        #backs[back].report()
        print(f"{back} -> {backs[back].get_profit()}")
    
    print()
    profs.append(profit)



Selected: 63
Current profit: 5.0999
Minutes=15 Window=10 Step=1 | 101.70597967714077 {'ltceur': 0.011623296645416337, 'ltcusd': 0.02971201855293026, 'ethusd': -0.025293287502104803, 'btcusd': -0.005435252527686549}
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('decisiontreeclassifier',
                 DecisionTreeClassifier(random_state=42))])
ethusd -> 5.0999

Current profit: 4.6586
Minutes=15 Window=20 Step=1 | 103.84625373091797 {'ltceur': 0.023387728644729575, 'ltcusd': 0.007805766658740209, 'ethusd': -0.002256458522902725, 'btcusd': -0.001400214792523935}
Pipeline(steps=[('normalizer', Normalizer()),
                ('pipeline',
                 Pipeline(steps=[('s',
                                  RFECV(estimator=DecisionTreeClassifier(random_state=10),
                                        scoring='recall')),
                                 ('m',
                                  DecisionTreeClassifier(random_state=42))]))])
ethusd -> 4.6586

Curre

Pipeline(steps=[('normalizer', Normalizer()),
                ('pipeline',
                 Pipeline(steps=[('s',
                                  RFECV(estimator=DecisionTreeClassifier(random_state=10),
                                        scoring='recall')),
                                 ('m',
                                  <catboost.core.CatBoostClassifier object at 0x7f884c0e6220>)]))])
ethusd -> 0.5031

Current profit: 0.4307
Minutes=1 Window=40 Step=1 | 100.6108304370236 {'ltceur': -7.677924078324275e-05, 'ltcusd': -0.011647717258909092, 'ethusd': 0.02289788248878743, 'btcusd': 0.006407103060061483}
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('decisiontreeclassifier',
                 DecisionTreeClassifier(random_state=42))])
ethusd -> 0.4307

Current profit: 0.4245
Minutes=15 Window=40 Step=1 | 100.76976136738162 {'ltceur': 0.011045296446080839, 'ltcusd': -0.005392501284910931, 'ethusd': -0.0056043979550364715, 'btcusd': 0.007272517660377394

In [102]:
winner = scoreboard[0]['result']
print_result(winner)
model_path = f'model/winner'
dump(winner, model_path)
print()
print(f"{np.average(profs)} ")

Minutes=15 Window=10 Step=1 | 101.70597967714077 {'ltceur': 0.011623296645416337, 'ltcusd': 0.02971201855293026, 'ethusd': -0.025293287502104803, 'btcusd': -0.005435252527686549}
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('decisiontreeclassifier',
                 DecisionTreeClassifier(random_state=42))])

1.1394095238095234 


In [103]:
model = scoreboard[0]['result']['model']

In [104]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('decisiontreeclassifier',
                 DecisionTreeClassifier(random_state=42))])